<a href="https://colab.research.google.com/github/antoniod621/CRUD-App-Students/blob/main/4_join.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#to solve this go and read the class lecture on map reduce where we discussed a trivial join example.
#here the join key is playerid and year id

In [1]:
#install mrjob
!pip install mrjob

     |████████████████████████████████| 439 kB 4.1 MB/s 


In [2]:
from google.colab import files

# upload dataset
uploaded = files.upload()
for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving Batting.csv to Batting.csv
Saving Salaries.csv to Salaries.csv
User uploaded file "Batting.csv" with length 6488747 bytes
User uploaded file "Salaries.csv" with length 700024 bytes


In [ ]:
from google.colab import files

# upload mrjob.conf under current directory(/content/mrjob.conf)
uploaded = files.upload()
for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

In [45]:
%%file 4_join.py
#write the top pragma string to save this file in local colab drive as 3_days.py
from mrjob.job import MRJob
from mrjob.step import MRStep

'''
Join the batting and salaries data for Barry Bonds per year.

The output should be the combined CSV string of batting and salaries data (one per year).

Use 'join' as the key for the final output in the reducer.

E.g:
"join"  "bondsba01,1986,1,PIT,NL,113,413,72,92,26,3,16,48,36,7,65,102,2,2,2,2,41986,PIT,NL,bondsba01,60000"

Schema:
Salaries: yearID  teamID  lgID  playerID  salary
Batting: playerID yearID  stint teamID  lgID  G AB  R H 2B  3B  HR  RBI SB  CS  BB  SO

Hints: 
Use split to split the CSV lines (e.g. s = line.split(','))
Both files are sent to the mapper. Use the length of the lines to determine which is which.
'''



class Join(MRJob):
  SORT_VALUES = True
  def mapper(self, _, line):
        fields = line.split(",")
        if len(fields) == 5:
          if fields[3] == "bondsba01":
            yield fields[0], ('s',fields[0], fields[1], fields[2], fields[3],
                              fields[4])
          else:
            if fields[0] == "bondsba01":
              yield fields[0], ('b', fields[0]. fields[1]. fields[2], fields[3],
                                fields[4]. fields[5]. fields[6], fields[7],
                                fields[8]. fields[9]. fields[10], fields[11],
                                fields[12]. fields[13]. fields[14], fields[15],
                                fields[16]. fields[17]. fields[18], fields[19],
                                fields[20]. fields[21]. fields[22], fields[23])
                        
  def reducer(self, year, lines):
     # do something
     for line in lines:
       yield year, str(line)
          

  def steps(self):
      return [
              MRStep(mapper = self.mapper, reducer = self.reducer)
      ]

if __name__ == '__main__':
    Join.run()

Overwriting 4_join.py


## Step 1. Run MR program locally(Colab)

In [46]:
#write the command to run it first to check for error
%%timeit
!python 4_join.py Batting.csv Salaries.csv > 4_join.out

No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory /tmp/4_join.root.20220401.021703.374664
Running step 1 of 1...
job output is in /tmp/4_join.root.20220401.021703.374664/output
Streaming final output from /tmp/4_join.root.20220401.021703.374664/output...
Removing temp directory /tmp/4_join.root.20220401.021703.374664...
No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory /tmp/4_join.root.20220401.021704.077381
Running step 1 of 1...
job output is in /tmp/4_join.root.20220401.021704.077381/output
Streaming final output from /tmp/4_join.root.20220401.021704.077381/output...
Removing temp directory /tmp/4_join.root.20220401.021704.077381...
No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory /tmp/4_join.root.20220401.021704.696999
Running step 1 of 1...
job output is in /tmp/4_join.root.20220401.02

In [47]:
!cat 4_join.out

"1992"	"['s', '1992', 'PIT', 'NL', 'bondsba01', '4800000']"
"1993"	"['s', '1993', 'SFN', 'NL', 'bondsba01', '4516666']"
"1994"	"['s', '1994', 'SFN', 'NL', 'bondsba01', '5166666']"
"1995"	"['s', '1995', 'SFN', 'NL', 'bondsba01', '8166666']"
"1996"	"['s', '1996', 'SFN', 'NL', 'bondsba01', '8416667']"
"1997"	"['s', '1997', 'SFN', 'NL', 'bondsba01', '8666667']"
"1998"	"['s', '1998', 'SFN', 'NL', 'bondsba01', '8916667']"
"1999"	"['s', '1999', 'SFN', 'NL', 'bondsba01', '9381057']"
"2000"	"['s', '2000', 'SFN', 'NL', 'bondsba01', '10658826']"
"2001"	"['s', '2001', 'SFN', 'NL', 'bondsba01', '10300000']"
"2002"	"['s', '2002', 'SFN', 'NL', 'bondsba01', '15000000']"
"2003"	"['s', '2003', 'SFN', 'NL', 'bondsba01', '15500000']"
"1986"	"['s', '1986', 'PIT', 'NL', 'bondsba01', '60000']"
"1987"	"['s', '1987', 'PIT', 'NL', 'bondsba01', '100000']"
"1988"	"['s', '1988', 'PIT', 'NL', 'bondsba01', '220000']"
"1989"	"['s', '1989', 'PIT', 'NL', 'bondsba01', '360000']"
"1990"	"['s', '1990', 'PIT', 'NL', 'bonds

## Step 2. Run MR program on AWS EMR

### Please copy  Batting.csv and Salaries.csv into your S3 bucket.

In [ ]:
#run and save output as 4_join.out
#note that you can pass two files -- in any order to the mrjob
#you might first check for errors before cascading the output it to 4_join.out
%%timeit
!python 4_join.py -r emr s3://<s3 url of the batting dataset> s3://<s3 url of the salaries dataset> --cloud-tmp-dir=s3://<s3 url of tmp directory> --cluster-id=<cluster ID> --conf-path <mrjob.conf file path in colab> > 4_join.out

In [ ]:
!cat 4_join.out

In [ ]:
def test_count():
    f = open('4_join.out')
    lines = f.readlines()
    f.close()
    
    found_line = False
    for  line in lines:
        if line[:-1] in [
            '"join"\t"bondsba01,1986,1,PIT,NL,113,413,72,92,26,3,16,48,36,7,65,102,2,2,2,2,41986,PIT,NL,bondsba01,60000"',
            '"join"\t"bondsba01,1986,1,PIT,NL,113,413,72,92,26,3,16,48,36,7,65,102,2,2,2,2,4,1986,PIT,NL,bondsba01,60000"']:
            found_line = True
        
    if not found_line:
        assert False

    
    assert len(lines) == 22

test_count()